# Plot boundaries on the Earth

Ce notebook est un résumé du notebook situé ici 

https://chart-studio.plotly.com/~empet/14813/heatmap-plot-on-a-spherical-map/#/

et se concentre sur le traçage des frontières (plus généralement d'un polygone) sur la sphère.

Il nécessite l'installation de **mpl_toolkits** qui peut être assez complexe.

Sous linux, sous réserve que la librairie GEOS soit installée, il suffit d'effectuer les commandes suivantes (le dossier à télécharger est assez lourd et la procédure peut prendre quelques minutes) :

En cas de problème, voir les détails ici :

https://matplotlib.org/basemap/users/installing.html .


In [ ]:
import numpy as np     
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.path import Path
import plotly.graph_objects as go
import IPython
from numpy import pi, cos, sin
import gstlearn as gl 

In [ ]:

def d2r(x):
    return x * np.pi / 180.

def mapping_map_to_sphere(lon, lat, radius=6371.):
    #this function maps the points of coords (lon, lat) to points onto the  sphere of radius radius
    
    lon=np.array(lon, dtype=np.float64)
    lat=np.array(lat, dtype=np.float64)
    lon=d2r(lon)
    lat=d2r(lat)
    xs=radius*cos(lon)*cos(lat)
    ys=radius*sin(lon)*cos(lat)
    zs=radius*sin(lat)
    return xs, ys, zs


# Functions converting coastline/country polygons to lon/lat traces
def polygons_to_traces(poly_paths, N_poly):
    ''' 
    pos arg 1. (poly_paths): paths to polygons
    pos arg 2. (N_poly): number of polygon to convert
    '''
    # init. plotting list
    lons=[]
    lats=[]

    for i_poly in range(N_poly):
        poly_path = poly_paths[i_poly]
        
        # get the Basemap coordinates of each segment
        coords_cc = np.array(
            [(vertex[0],vertex[1]) 
             for (vertex,code) in poly_path.iter_segments(simplify=False)]
        )
        
        # convert coordinates to lon/lat by 'inverting' the Basemap projection
        lon_cc, lat_cc = m(coords_cc[:,0],coords_cc[:,1], inverse=True)
    
        
        lats.extend(lat_cc.tolist()+[None]) 
        lons.extend(lon_cc.tolist()+[None])
        
       
    return lons, lats


# Function generating coastline lon/lat 
def get_coastline_traces():
    #poly_paths = m.drawcoastlines().get_paths() # coastline polygon paths
    poly_paths = [Path(np.array(i)) for i in m.coastsegs]
    N_poly = 91  # use only the 91st biggest coastlines (i.e. no rivers)
    cc_lons, cc_lats= polygons_to_traces(poly_paths, N_poly)
    return cc_lons, cc_lats

# Function generating country lon/lat 
def get_country_traces():
    fig= plt.figure()
    poly_paths = m.drawcountries().get_paths() # country polygon path
    plt.close()
    poly_paths = [Path(np.array(i)) for i in m.cntrysegs]
    N_poly = len(poly_paths)  # use all countries
    country_lons, country_lats= polygons_to_traces(poly_paths, N_poly)
    return country_lons, country_lats








In [ ]:
gl.variety_define(1)

mesh = gl.MeshSpherical()
mesh.reset(None,None,triswitch = "-r5",verbose=False)

X = mesh.getCoordinates(0)
Y = mesh.getCoordinates(1)




In [ ]:

model = gl.Model.createFromParam(gl.ECov.BESSEL_K,range=1500,param=1) #penser à règler range dans shiftop
S=gl.ShiftOpCs(mesh,model)
whitenoise = gl.ut_vector_simulate_gaussian(mesh.getNApices())
result = gl.VectorDouble(mesh.getNApices())
Q = gl.PrecisionOpCs(S,model.getCova(0),gl.EPowerPT.MINUSHALF,verbose=True)
Q.eval(whitenoise,result)


db = gl.Db.create()
db["long"]=X
db["lat"]=Y
db.setLocators(["long","lat"],gl.ELoc.X)


In [ ]:

meshes = np.array(mesh.getMeshes()).reshape([mesh.getNMeshes(),3])-1
xs,ys,zs = mapping_map_to_sphere(db["long"][:,0],db["lat"][:,0])


cscale = [[0.0, '#313695'],
 [0.07692307692307693, '#3a67af'],
 [0.15384615384615385, '#5994c5'],
 [0.23076923076923078, '#84bbd8'],
 [0.3076923076923077, '#afdbea'],
 [0.38461538461538464, '#d8eff5'],
 [0.46153846153846156, '#d6ffe1'],
 [0.5384615384615384, '#fef4ac'],
 [0.6153846153846154, '#fed987'],
 [0.6923076923076923, '#fdb264'],
 [0.7692307692307693, '#f78249'],
 [0.8461538461538461, '#e75435'],
 [0.9230769230769231, '#cc2727'],
 [1.0, '#a50026']]

surf = go.Mesh3d(
        x=xs,
        y=ys,
        z=zs,
        colorbar_title='z',
        colorscale=cscale,
        # Intensity of each vertex, which will be interpolated and color-coded
        intensity=np.array(result),
        # i, j and k give the vertices of triangles
        # here we represent the 4 triangles of the tetrahedron surface
        i=meshes[:,0],
        j=meshes[:,1],
        k=meshes[:,2],
        name='y',
        showscale=True
    )

In [ ]:
m = Basemap() 



# Get list of of coastline, country, and state lon/lat 

cc_lons, cc_lats=get_coastline_traces()
country_lons, country_lats=get_country_traces()

#concatenate the lon/lat for coastlines and country boundaries:
lons=cc_lons+[None]+country_lons
lats=cc_lats+[None]+country_lats

xs, ys, zs=mapping_map_to_sphere(np.array(lons),np.array(lats), radius=6371*1.01)# here the radius is slightly greater than 1 
                                                         #to ensure lines visibility; otherwise (with radius=1)
                                                         # some lines are hidden by contours colors


boundaries=dict(type='scatter3d',
               x=xs,
               y=ys,
               z=zs,
               mode='lines',
               line=dict(color='black', width=1)
              )

fig = go.Figure(data = [surf,boundaries])
fig.show()

In [ ]:
u = np.where([i==None for i in lons])[0]
u = np.insert(u,0,-1)
long  = [lons[u[i]+1:u[i+1]] for i in range(len(u)-1)]
lat =  [lats[u[i]+1:u[i+1]] for i in range(len(u)-1)]

In [ ]:
P = gl.Polygons()
p = []
for i,ind in enumerate(zip(long,lat)):
    if(len(ind[0])>0):
        v = gl.PolySet.create()
        v.setX(ind[0])
        v.setY(ind[1])
        p += [v]
        P.addPolySet(v)
        
P.dumpToNF("../../../doc/data/boundaries/world.poly")

## Test 

In [ ]:
poly = gl.Polygons.createFromNF("../../../doc/data/boundaries/world.poly")

In [ ]:
#poly = P
#poly = gl.Polygons.createFromNF("../../../doc/data/boundaries/world.poly")
poly = P
X = list()
Y = list()

for i in range(P.getPolySetNumber()):
    a = list(P.getX(i))
    b = list(P.getY(i))
    #a.pop()
    #b.pop()
    X.extend(a + [None])
    Y.extend(b + [None])
    
X = np.array(X)
Y = np.array(Y)

#xs,ys,zs = mapping_map_to_sphere(X,Y)

In [ ]:
xs, ys, zs=mapping_map_to_sphere(np.array(X),np.array(Y), radius=6371*1.01)# here the radius is slightly greater than 1 
                                                         #to ensure lines visibility; otherwise (with radius=1)
                                                         # some lines are hidden by contours colors


boundaries=dict(type='scatter3d',
               x=xs,
               y=ys,
               z=zs,
               mode='lines',
               line=dict(color='black', width=1)
              )

fig = go.Figure(data = [boundaries])
fig.show()

In [ ]:
import gstlearn as gl


In [ ]:
A = gl.MatrixRectangular(2,3)
A.setValues()

In [ ]:
matS = S.getS()
import scipy as sc
from scipy.sparse import *


In [ ]:
Bmat.toarray()[:,0][np.where(Bmat.toarray()[:,0]!=0)[0]]

In [ ]:
Btr = gl.csToTriplet(matS)
Bmat=sc.sparse.csc_matrix((np.array(Btr.values), (np.array(Btr.rows), np.array(Btr.cols))),
                          shape=(Btr.nrows,Btr.ncols))